# Prices API Demo
這份筆記示範如何從 FastAPI `GET /prices/` 端點取得資料，將 JSON payload 轉回 `DataFrame` 再保留和 `FinlabDataFrame` 相容的結構。

In [2]:
import os
from datetime import datetime

import pandas as pd
import requests


In [ ]:
API_BASE = os.getenv("PRICES_API_BASE", "http://127.0.0.1:8000")
PRICES_ENDPOINT = f"{API_BASE}/prices/"

params = {
    "exchange": "binance",
    "timeframe": "4h",
    "field": "close",
    "start": datetime(2025, 5, 1).isoformat() + "Z",
    "end": datetime(2025, 5, 15).isoformat() + "Z",
}
vol = {
    "exchange": "binance",
    "timeframe": "4h",
    "field": "volumn",
    "start": datetime(2025, 5, 1).isoformat() + "Z",
    "end": datetime(2025, 5, 15).isoformat() + "Z",
}
params


{'exchange': 'binance',
 'timeframe': '4h',
 'field': 'close',
 'start': '2025-05-01T00:00:00Z',
 'end': '2025-05-15T00:00:00Z'}

In [4]:
response = requests.get(PRICES_ENDPOINT, params=params)
response.raise_for_status()
payload = response.json()["context"]
payload.keys()


dict_keys(['meta', 'data'])

In [10]:
import sys
from pathlib import Path

repo_root = Path.cwd().parent  # adjust if the notebook isn’t under notebooks/
sys.path.insert(0, str(repo_root))

from backend.app.services.finlab_price import FinlabDataFrame, FinlabFrameMeta


In [12]:
from backend.app.services.finlab_price import FinlabDataFrame, FinlabFrameMeta

data = pd.DataFrame(payload["data"])
data["ts"] = pd.to_datetime(data["ts"])
frame = data.set_index("ts").sort_index()
frame = FinlabDataFrame(frame)


In [14]:
cond1 = frame > frame.average(5)
cond1

,1000CAT/USDT,1000CHEEMS/USDT,1000SATS/USDT,1INCH/USDT,1MBABYDOGE/USDT,AAVE/USDT,ACA/USDT,ACE/USDT,ACH/USDT,ACM/USDT,...,XVG/USDT,XVS/USDT,YFI/USDT,YGG/USDT,ZEC/USDT,ZEN/USDT,ZIL/USDT,ZK/USDT,ZRO/USDT,ZRX/USDT
ts,,,,,,,,,,,,,,,,,,,,,
2025-05-01 00:00:00+00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2025-05-01 04:00:00+00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2025-05-01 08:00:00+00:00,True,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True
2025-05-01 12:00:00+00:00,True,False,False,True,True,True,True,False,True,False,...,True,False,True,True,False,True,True,True,False,True
2025-05-01 16:00:00+00:00,True,False,False,True,True,True,False,False,True,True,...,True,True,False,False,False,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-14 08:00:00+00:00,False,False,True,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
2025-05-14 12:00:00+00:00,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2025-05-14 16:00:00+00:00,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False


In [5]:
meta = payload.get("meta")
meta


{'exchange': 'binance',
 'timeframe': '4h',
 'field': 'close',
 'symbols': ['BTC/USDT', 'ETH/USDT'],
 'start': '2025-05-01T00:00:00+00:00',
 'end': '2025-05-15T00:00:00+00:00'}